In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("hw_pyspark_env_demo")
         .master("local[*]")
         .config("spark.sql.shuffle.partitions", "8")
         .config("spark.driver.memory", "4g")
         .getOrCreate())
sc = spark.sparkContext

print("Spark version:", sc.version)
print("Master:", sc.master)
print("Shuffle partitions:", spark.conf.get("spark.sql.shuffle.partitions"))
print("Driver memory:", spark.conf.get("spark.driver.memory"))

spark.createDataFrame([(1,"test")], ["id","msg"]).show()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/07 21:40:55 WARN Utils: Your hostname, Samuels-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.207 instead (on interface en0)
25/11/07 21:40:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/07 21:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1
Master: local[*]
Shuffle partitions: 8
Driver memory: 4g


[Stage 0:>                                                          (0 + 1) / 1]

+---+----+
| id| msg|
+---+----+
|  1|test|
+---+----+



In [2]:
# Getting dataset from URL
!mkdir -p data

TRACKS_URL = "https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset/resolve/main/dataset.csv?download=true"

!wget -q -O data/spotify_tracks.csv "$TRACKS_URL"
!ls -lh data

total 39296
-rw-r--r--@ 1 sbleo  staff    19M Nov  7 21:41 spotify_tracks.csv


In [3]:
# Cleaning commas in dataset
!grep -vi '".*,.*"' data/spotify_tracks.csv > data/clean_spotify_tracks.csv

In [4]:
# Reading dataset into "tracks"
tracks = (spark.read.option("header", True).option("inferSchema", True)
          .csv("data/clean_spotify_tracks.csv")
          .select("track_id", "artists", "album_name", "track_name", "popularity", "duration_ms", "explicit", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature", "track_genre"))

print("Tracks schema:"); tracks.printSchema()
tracks.show(5, truncate=False)

Tracks schema:
root
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- track_genre: string (nullable = true)

+----------------------+----------------------+------------------------------------------------------+--------------------------+----------+-----------+

In [5]:
tracks.describe()

DataFrame[summary: string, track_id: string, artists: string, album_name: string, track_name: string, popularity: string, duration_ms: string, danceability: string, energy: string, key: string, loudness: string, mode: string, speechiness: string, acousticness: string, instrumentalness: string, liveness: string, valence: string, tempo: string, time_signature: string, track_genre: string]

In [6]:
# Most Frequent Danceability
import pyspark.sql.functions as F
freq_danc = (tracks.groupBy("danceability")
               .agg(F.count("*").alias("freq"))
               .orderBy(F.desc("freq"))
               .limit(50))
freq_danc.show(50, truncate=False)

+------------+----+
|danceability|freq|
+------------+----+
|0.647       |406 |
|0.609       |331 |
|0.579       |330 |
|0.685       |324 |
|0.602       |316 |
|0.689       |305 |
|0.524       |303 |
|0.598       |294 |
|0.586       |294 |
|0.626       |291 |
|0.631       |290 |
|0.582       |289 |
|0.56        |289 |
|0.607       |287 |
|0.534       |282 |
|0.576       |281 |
|0.545       |281 |
|0.532       |274 |
|0.593       |273 |
|0.627       |272 |
|0.588       |272 |
|0.568       |272 |
|0.687       |270 |
|0.714       |270 |
|0.795       |268 |
|0.671       |268 |
|0.573       |267 |
|0.569       |266 |
|0.639       |266 |
|0.623       |264 |
|0.535       |263 |
|0.653       |261 |
|0.616       |260 |
|0.603       |259 |
|0.497       |259 |
|0.503       |258 |
|0.661       |257 |
|0.596       |257 |
|0.533       |256 |
|0.637       |255 |
|0.679       |255 |
|0.521       |255 |
|0.705       |254 |
|0.546       |253 |
|0.629       |253 |
|0.543       |252 |
|0.601       |252 |


In [7]:
import pyspark.sql.functions as F

genre_stats = (tracks.groupBy("track_genre")
               .agg(
                   F.count("*").alias("track_count"),
                   F.avg("popularity").alias("avg_popularity"),
                   F.avg("danceability").alias("avg_danceability"),
                   F.avg("energy").alias("avg_energy"),
                   F.avg("duration_ms").alias("avg_duration_ms")
               )
               .orderBy(F.desc("avg_popularity"))
               .limit(20))

print("Top 20 Genres by Average Popularity:")
genre_stats.show(20, truncate=False)

Top 20 Genres by Average Popularity:
+-----------------+-----------+------------------+-------------------+-------------------+------------------+
|track_genre      |track_count|avg_popularity    |avg_danceability   |avg_energy         |avg_duration_ms   |
+-----------------+-----------+------------------+-------------------+-------------------+------------------+
|pop-film         |894        |58.91834451901566 |0.5985469798657724 |0.604239261744966  |282771.725950783  |
|k-pop            |932        |56.75858369098712 |0.650704935622317  |0.680214592274679  |253326.51502145923|
|chill            |974        |53.52669404517454 |0.6647104722792606 |0.42651940451745357|169359.5246406571 |
|sad              |946        |52.471458773784356|0.6935105708245243 |0.4586343551797044 |153646.19873150106|
|grunge           |970        |49.636082474226804|0.4583123711340208 |0.8038402061855668 |234538.24020618555|
|indian           |929        |49.06673842841765 |0.593657696447793  |0.56153498385